In [21]:
import numpy as np
import pandas as pd
from sample_reader.sample_reader import SampleReader

In [2]:
samples_fv = SampleReader().read('AbVitro/flu_time_course/FV/', ['25'])
samples_gmc = SampleReader().read('AbVitro/flu_time_course/GMC/', ['8'])
samples_ido = SampleReader().read('AbVitro/flu_time_course/IDO/')
samples_age = SampleReader().read('age/')
samples_paired = SampleReader().read('AbVitro/paired/')

In [3]:
from shm_kmer_model_estimator.shm_kmer_model_estimator import ShmKmerModelEstimator

In [4]:
estimator = ShmKmerModelEstimator()
est_model = estimator.estimate_models_of_one_type((samples_age,
                                                   samples_ido,
                                                   samples_fv,
                                                   samples_gmc,
                                                   samples_paired
                                                  ),
                                                  chains=['IGH'])

NoKNeighbours: IGH


/Users/andrewbzikadze/anaconda/envs/python2.7/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


Trivial: IGH


In [9]:
import shm_kmer_model.shm_kmer_model as shm_kmer_model

In [7]:
#est_model['NoKNeighbours']['IGH'].to_csv('/Users/andrewbzikadze/chihua/datasets/shm_model_no_kneighbours.csv', na_rep='NaN')
#est_model['Trivial']['IGH'].to_csv('/Users/andrewbzikadze/chihua/datasets/shm_model_trivial.csv', na_rep='NaN')

In [15]:
def apply_to_datasets(datasets, func):
    result = dict.fromkeys(datasets)
    for strategy in datasets:
        result[strategy] = dict.fromkeys(datasets[strategy])
        for chain_type in datasets[strategy]:
            result[strategy][chain_type] = \
            func(datasets[strategy][chain_type].get_raw_dataset())
    return result

In [16]:
apply_to_datasets(est_model, (lambda x: x.isnull().any(axis=1).mean()))

{'NoKNeighbours': {'IGH': 0.1611328125}, 'Trivial': {'IGH': 0.1201171875}}

In [17]:
apply_to_datasets(est_model, (lambda x: x['success_optim_beta'].mean()))

{'NoKNeighbours': {'IGH': 0.8505859375}, 'Trivial': {'IGH': 0.9228515625}}

In [18]:
apply_to_datasets(est_model, lambda x: x['success_optim_dir'].mean())

{'NoKNeighbours': {'IGH': 0.98828125}, 'Trivial': {'IGH': 0.994140625}}

In [22]:
def sd_beta(x):
    a, b = x['beta_shape1'], x['beta_shape2']
    return np.sqrt(a * b / ((a + b)**2 * (a + b + 1)))

In [23]:
apply_to_datasets(est_model, sd_beta)
None

In [28]:
def sd_dir(x):
    alpha = x.ix[:, np.arange(2, 5)]
    alpha_sum = np.sum(alpha)
    denominator = alpha_sum * np.sqrt(alpha_sum + 1)
    return np.sqrt(alpha * (alpha_sum - alpha)) / denominator

In [25]:
apply_to_datasets(est_model, sd_dir)
None

In [26]:
apply_to_datasets(est_model, (lambda x: np.nanstd(np.array(x.ix[:,0:2].T / np.sum(x.ix[:,0:2], axis=1)).T - \
                                        np.array(x.ix[:,7:9]))))

{'NoKNeighbours': {'IGH': 0.017479290381984831},
 'Trivial': {'IGH': 0.014590950009082691}}

In [27]:
apply_to_datasets(est_model, (lambda x: np.nanstd(np.array(x.ix[:,2:5].T / np.sum(x.ix[:,2:5], axis=1)).T - \
                                        np.array(x.ix[:,9:12]))))

{'NoKNeighbours': {'IGH': 0.033168516056584725},
 'Trivial': {'IGH': 0.02091559250794919}}